In [267]:
import pandas as pd
import numpy as np
import warnings

import tensorflow as tf 
from tensorflow.keras.layers import SimpleRNN, GRU, LSTM, Embedding, Dense
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizerFast

import spacy
from spacy.lang.en.stop_words import STOP_WORDS



# EDA & Data Cleaning

In [214]:
df = pd.read_csv('Data/train.csv')
df.head(15)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [215]:
df.describe(include='all')

,id,keyword,location,text,target
count,7613.000000,7552,5080,7613,7613.00000
unique,NaN,221,3341,7503,NaN
top,NaN,fatalities,USA,11-Year-Old Boy Charged With Manslaughter of T...,NaN
freq,NaN,45,104,10,NaN
mean,5441.934848,NaN,NaN,NaN,0.42966
std,3137.116090,NaN,NaN,NaN,0.49506
min,1.000000,NaN,NaN,NaN,0.00000
25%,2734.000000,NaN,NaN,NaN,0.00000
50%,5408.000000,NaN,NaN,NaN,0.00000
75%,8146.000000,NaN,NaN,NaN,1.00000


Column 'id' is not going to help us with the prediction, we can drop it straight away.
Furthermore, we will look straight away into getting the prediction purely from the tweet content as we have a lot of missing value, so we'll drop the keyword & location column.

It seems like we have some duplicate within the text column too, we'll look into that. Do they have the same target? If they do we'll remove them to have just one of each text, if they don't we need to look into why they don't.

In [216]:
# Dropping aforementionned columns
df.drop(['id','keyword','location'],axis=1,inplace=True)

# Keep = False is to avoid that it keeps the first instance of each duplicate which would result in duplicates remaining
#Looking at duplicated text
df[df['text'].duplicated(keep=False)].sort_values('text')

,text,target
4290,#Allah describes piling up #wealth thinking it...,0
4299,#Allah describes piling up #wealth thinking it...,0
4312,#Allah describes piling up #wealth thinking it...,1
6363,#Bestnaijamade: 16yr old PKK suicide bomber wh...,1
6373,#Bestnaijamade: 16yr old PKK suicide bomber wh...,1
...,...,...
6091,that horrible sinking feeling when youÛªve be...,1
6103,that horrible sinking feeling when youÛªve be...,0
6094,that horrible sinking feeling when youÛªve be...,0
5641,wowo--=== 12000 Nigerian refugees repatriated ...,0


Looks like some have the same target, some don't. If they have the same target, we'll clean up such that we have just one of each, if they don't we'll remove them altogether

In [217]:
df['index'] = df.index # Will help to make list of originals index to pick up the data after

aggs = {'target': 'mean',
         'index':lambda x: list(x)} # Keep track of the index 

grouped_df = df[df['text'].duplicated(keep=False)].groupby('text').agg(aggs) 

# if target mean is not equal to 0 or 1 then the target is not consistent throughout the observations, let's see which one they are
dif_targets = grouped_df[(grouped_df.target!=1) & (grouped_df.target!=0)].reset_index()
dif_targets

,text,target,index
0,#Allah describes piling up #wealth thinking it...,0.333333,"[4290, 4299, 4312]"
1,#foodscare #offers2go #NestleIndia slips into ...,0.666667,"[4221, 4239, 4244]"
2,.POTUS #StrategicPatience is a strategy for #G...,0.750000,"[2830, 2831, 2832, 2833]"
3,CLEARED:incident with injury:I-495 inner loop...,0.666667,"[4597, 4605, 4618]"
4,Caution: breathing may be hazardous to your he...,0.500000,"[4232, 4235]"
5,He came to a land which was engulfed in tribal...,0.333333,"[3240, 3243, 3248, 3251, 3261, 3266]"
6,Hellfire is surrounded by desires so be carefu...,0.333333,"[4285, 4305, 4313]"
7,Hellfire! We donÛªt even want to think about ...,0.500000,"[4306, 4320]"
8,I Pledge Allegiance To The P.O.P.E. And The Bu...,0.500000,"[1214, 1365]"
9,In #islam saving a person is equal in reward t...,0.500000,"[6614, 6616]"


We could take fix the target manually but given there is only 18 of them and some are ambiguous, we will proceed with removing them

In [218]:
# We'll save down the column of index as a list for us to drop them
to_drop_dif_targets = dif_targets['index'].tolist()
to_drop_dif_targets = [item for sublist in to_drop_dif_targets for item in sublist] #flatten nested list

# Dropping duplicates with  target
shape = df.shape[0]
df.drop(to_drop_dif_targets,inplace=True)
dropped =  shape - df.shape[0]

print(f"We dropped {dropped} observations")


# Cleaning up
df.drop('index',axis=1,inplace=True) # no longer need that column


We dropped 55 observations


In [219]:
# Let's now look at the one that have the same target
# For those we'll keep only one of each to avoid any bias
same_targets = grouped_df[(grouped_df.target==1) | (grouped_df.target==0)].reset_index()

print(same_targets.shape)
same_targets.head(5)

(51, 3)


,text,target,index
0,#Bestnaijamade: 16yr old PKK suicide bomber wh...,1.0,"[6363, 6366, 6373, 6377, 6378, 6392]"
1,#KCA #VoteJKT48ID 12News: UPDATE: A family of ...,1.0,"[2822, 2828]"
2,#Myanmar Displaced #Rohingya at #Sittwe point...,1.0,"[2816, 2841]"
3,#Newswatch: 2 vehicles collided at Lock and La...,1.0,"[1704, 1725]"
4,#SigAlert: North &amp; Southbound 133 closed b...,1.0,"[3790, 3795]"


In [220]:
# Dropping duplicates above keeping only one instance of each tweet

shape = df.shape[0]

df.drop_duplicates(keep='first',inplace=True)

dropped =  shape - df.shape[0]
print("We dropped {} observations".format(dropped))

We dropped 73 observations


Let's have a final look at the dataset before we proceed to verify everything seems cleaned up

In [221]:
df.describe(include='all').iloc[0:2]

,text,target
count,7485,7485.0
unique,7485,NaN


In [222]:
df['target'].value_counts(normalize=True) # Checking for balanced classes

0    0.574081
1    0.425919
Name: target, dtype: float64

In [223]:
df.reset_index(inplace=True,drop=True) # deals with some missing index


for i in range(20):
    print(df['text'][i]) # allows to see the complete text

Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all
Forest fire near La Ronge Sask. Canada
All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected
13,000 people receive #wildfires evacuation orders in California 
Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school 
#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires
#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas
I'm on top of the hill and I can see a fire in the woods...
There's an emergency evacuation happening now in the building across the street
I'm afraid that the tornado is coming to our area...
Three people died from the heat wave so far
Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA DO FVCK #flooding
#raining #flooding #Florida 

This already looks better, no more duplicated values. There is no missing values so we won't need to remove any more observations. 

The dataset is fairly balanced - about 42% of the tweets indicate a disaster.

Let's clean up the texts a bit. There's a lot of links, but those will get out by themselves once we limit the vocabulary size as they are all unique.  

Furthermore, we'll remove ponctuation, clean up if there's any double space, put everything in lowercase and remove the stop words, along with lemmatising the words.

In [224]:
nlp = spacy.load("en_core_web_sm") # Loading english language elements from spacy

df['text_clean'] = df["text"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" "))
df['text_clean'] = df["text_clean"].apply(lambda x: x.replace(" +"," ").lower().strip())
df['text_clean'] = df["text_clean"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) and (token.text not in STOP_WORDS)]))

df.head(20)


,text,target,text_clean
0,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake allah forgive
1,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,All residents asked to 'shelter in place' are ...,1,resident ask shelter place notify officer evac...
3,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfire evacuation order...
4,Just got sent this photo from Ruby #Alaska as ...,1,got send photo ruby alaska smoke wildfires pou...
5,#RockyFire Update => California Hwy. 20 closed...,1,rockyfire update california hwy 20 close dir...
6,#flood #disaster Heavy rain causes flash flood...,1,flood disaster heavy rain cause flash flooding...
7,I'm on top of the hill and I can see a fire in...,1,m hill fire wood
8,There's an emergency evacuation happening now ...,1,s emergency evacuation happen building street
9,I'm afraid that the tornado is coming to our a...,1,m afraid tornado come area


# Splitting data & tokenizing the texts

Even though the dataset is already split into train / test, the test one is only to submit to Kaggle and it does not have a target, making it complicated to evalue our model. 

We should thus train/test the train dataset to choose the best model. We'll also keep 10% for final testing to have an idea of how we expect our model to perform. 
The proportion are thus as follow:

    * Train: 75%

    * Validation: 15%
    
    * Test: 10%

We'll also both set a random state, to ensure we are always working with the same split, and stratify with respect to the target as we are working with a classification problem. 

However, before submitting to Kaggle, we'll retrain the model on the entire dataset as to not 'waste' any data.


In [225]:
X_train, X_test, Y_train, Y_test = train_test_split(df.text_clean, df.target ,test_size=0.25,random_state = 0, stratify = df.target)
X_test, X_valid, Y_test, Y_valid = train_test_split(X_test, Y_test ,test_size=0.6,random_state = 0, stratify = Y_test) # 0.6 * 0.25 = 0.15
 
print('Training:')
print(X_train.shape,Y_train.shape)
print('Validation:')
print(X_valid.shape, Y_valid.shape)
print('Testing:')
print(X_test.shape,Y_test.shape)


Training:
(5613,) (5613,)
Validation:
(1124,) (1124,)
Testing:
(748,) (748,)


In [226]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000, oov_token="<OOV>") # instanciate the tokenizer

tokenizer.fit_on_texts(X_train) # fitting tokenizer only on train dataset to prevent any data leakage

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_valid = tokenizer.texts_to_sequences(X_valid)


In [227]:
# Finding out the max length out of all the texts
MAX_LENGTH = len(max(X_train + X_test + X_valid, key =  lambda i: len(i) ))
print("The max length is ",MAX_LENGTH)

# Padding the sequences 
X_valid = tf.keras.preprocessing.sequence.pad_sequences(X_valid, padding="post", maxlen= MAX_LENGTH)
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, padding="post", maxlen= MAX_LENGTH)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, padding="post", maxlen= MAX_LENGTH)

The max length is  25


In [228]:
BATCH_SIZE = 124

train = tf.data.Dataset.from_tensor_slices((X_train,Y_train)).batch(BATCH_SIZE)
test = tf.data.Dataset.from_tensor_slices((X_test,Y_test.values)).batch(BATCH_SIZE)
valid = tf.data.Dataset.from_tensor_slices((X_test,Y_test.values)).batch(BATCH_SIZE)

for text, target in train.take(1):
  print(text, target)

tf.Tensor(
[[777   1  14 ...   0   0   0]
 [247  17 134 ...   0   0   0]
 [778  92   5 ...   0   0   0]
 ...
 [  1   1 910 ...   0   0   0]
 [ 51 310   1 ...   0   0   0]
 [104   1 566 ...   0   0   0]], shape=(124, 25), dtype=int32) tf.Tensor(
[1 1 1 1 0 1 0 0 1 0 0 0 1 0 1 0 1 1 1 1 0 1 1 0 1 1 1 0 1 0 0 0 0 0 1 1 0
 0 1 0 1 1 1 1 0 1 0 0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1
 0 0 1 1 1 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0 1 0 0 1 1 1 1 0 0
 0 1 0 1 1 0 0 0 0 0 0 0 0], shape=(124,), dtype=int64)


Now that the dataset is all pre-processed, we can move on to the model.

# Recurrent Neural Network model

In [229]:
vocab_size = len(tokenizer.word_index)

model = tf.keras.Sequential([
                  # Word Embedding layer           
                  Embedding(vocab_size, 64,input_shape=[MAX_LENGTH,]),
                  # Gobal average pooling
                  SimpleRNN(units=64, return_sequences=True), # maintains the sequential nature
                  SimpleRNN(units=32, return_sequences=False), # returns the last output
                  # Dense layers once the data is flat
                  Dense(16, activation='relu'),
                  Dense(8, activation='relu'),

                  # output layer with as many neurons as the number of classes
                  # for the target variable and softmax activation
                  Dense(1, activation="sigmoid")
])

In [230]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 25, 64)            1051008   
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 25, 64)            8256      
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 32)                3104      
                                                                 
 dense_9 (Dense)             (None, 16)                528       
                                                                 
 dense_10 (Dense)            (None, 8)                 136       
                                                                 
 dense_11 (Dense)            (None, 1)                 9         
                                                                 
Total params: 1,063,041
Trainable params: 1,063,041
No

In [231]:
optimizer= tf.keras.optimizers.Adam()

callbacks = EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True)

model.compile(optimizer=optimizer, 
              loss = tf.keras.losses.BinaryCrossentropy(), # bc binary case here
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [232]:
model.fit(train,
          epochs=25, 
          validation_data=valid,
          callbacks=callbacks)

Epoch 1/25
46/46 [==============================] - 12s 79ms/step - loss: 0.6170 - binary_accuracy: 0.6551 - val_loss: 0.5237 - val_binary_accuracy: 0.7754
Epoch 2/25
46/46 [==============================] - 2s 37ms/step - loss: 0.4635 - binary_accuracy: 0.7990 - val_loss: 0.4868 - val_binary_accuracy: 0.7660
Epoch 3/25
46/46 [==============================] - 2s 43ms/step - loss: 0.3912 - binary_accuracy: 0.8409 - val_loss: 0.5282 - val_binary_accuracy: 0.7607
Epoch 4/25
46/46 [==============================] - 3s 68ms/step - loss: 0.3271 - binary_accuracy: 0.8764 - val_loss: 0.5368 - val_binary_accuracy: 0.7848
Epoch 5/25
46/46 [==============================] - 2s 42ms/step - loss: 0.3160 - binary_accuracy: 0.8799 - val_loss: 0.5824 - val_binary_accuracy: 0.7674
Epoch 6/25
46/46 [==============================] - 2s 33ms/step - loss: 0.2944 - binary_accuracy: 0.8863 - val_loss: 0.5344 - val_binary_accuracy: 0.7968
Epoch 7/25
46/46 [==============================] - 2s 43ms/step - lo

In [233]:
model.save("models/model_RNN.h5") # model_simpleRNN_1 was done on batch size = 64


# GRU Model

In [234]:
model_gru = tf.keras.Sequential([
                  Embedding(vocab_size+1, 64, input_shape=[MAX_LENGTH,],name="embedding"),
                  GRU(units=64, return_sequences=True), # maintains the sequential nature
                  GRU(units=32, return_sequences=False), # returns the last output
                  Dense(16, activation='relu'),
                  Dense(8, activation='relu'),

                  Dense(1, activation="sigmoid")
])

In [235]:
model_gru.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 64)            1051072   
                                                                 
 gru_2 (GRU)                 (None, 25, 64)            24960     
                                                                 
 gru_3 (GRU)                 (None, 32)                9408      
                                                                 
 dense_12 (Dense)            (None, 16)                528       
                                                                 
 dense_13 (Dense)            (None, 8)                 136       
                                                                 
 dense_14 (Dense)            (None, 1)                 9         
                                                                 
Total params: 1,086,113
Trainable params: 1,086,113
No

In [236]:
optimizer= tf.keras.optimizers.Adam()


model_gru.compile(optimizer=optimizer, 
              loss = tf.keras.losses.BinaryCrossentropy(), # bc binary case here
              metrics=[tf.keras.metrics.BinaryAccuracy()])


callbacks = EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True)

In [237]:
model_gru.fit(train,
              epochs=20, 
              validation_data=valid,
              callbacks=callbacks
              )

Epoch 1/20
46/46 [==============================] - 13s 103ms/step - loss: 0.6833 - binary_accuracy: 0.5740 - val_loss: 0.6823 - val_binary_accuracy: 0.5749
Epoch 2/20
46/46 [==============================] - 4s 86ms/step - loss: 0.6798 - binary_accuracy: 0.5740 - val_loss: 0.6658 - val_binary_accuracy: 0.5789
Epoch 3/20
46/46 [==============================] - 4s 85ms/step - loss: 0.5318 - binary_accuracy: 0.7408 - val_loss: 0.4566 - val_binary_accuracy: 0.7968
Epoch 4/20
46/46 [==============================] - 4s 88ms/step - loss: 0.4302 - binary_accuracy: 0.8142 - val_loss: 0.4492 - val_binary_accuracy: 0.7861
Epoch 5/20
46/46 [==============================] - 4s 88ms/step - loss: 0.3928 - binary_accuracy: 0.8382 - val_loss: 0.4790 - val_binary_accuracy: 0.7981
Epoch 6/20
46/46 [==============================] - 4s 78ms/step - loss: 0.3766 - binary_accuracy: 0.8459 - val_loss: 0.5218 - val_binary_accuracy: 0.7941
Epoch 7/20
46/46 [==============================] - 7s 147ms/step - 

In [238]:
model_gru.save('models/model_GRU.h5')

## Long Short Term Memory Model

In [239]:
model_lstm = tf.keras.Sequential([
                  Embedding(vocab_size+1, 64, input_shape=[MAX_LENGTH],name="embedding"),
                  LSTM(units=64, return_sequences=True), # maintains the sequential nature
                  LSTM(units=32, return_sequences=False), # returns the last output
                  Dense(16, activation='relu'),
                  Dense(8, activation='relu'),
                  Dense(1, activation="sigmoid")
])

In [240]:
model_lstm.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 64)            1051072   
                                                                 
 lstm_2 (LSTM)               (None, 25, 64)            33024     
                                                                 
 lstm_3 (LSTM)               (None, 32)                12416     
                                                                 
 dense_15 (Dense)            (None, 16)                528       
                                                                 
 dense_16 (Dense)            (None, 8)                 136       
                                                                 
 dense_17 (Dense)            (None, 1)                 9         
                                                                 
Total params: 1,097,185
Trainable params: 1,097,185
No

In [241]:
optimizer= tf.keras.optimizers.Adam()

model_lstm.compile(optimizer=optimizer, 
              loss = tf.keras.losses.BinaryCrossentropy(), # bc binary case here
              metrics=[tf.keras.metrics.BinaryAccuracy()])

callbacks = EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 10, 
                                        restore_best_weights = True)

In [242]:
model_lstm.fit(train,
                epochs = 30,
                validation_data = valid,
                callbacks=callbacks)

Epoch 1/30
46/46 [==============================] - 12s 125ms/step - loss: 0.6594 - binary_accuracy: 0.5979 - val_loss: 0.5598 - val_binary_accuracy: 0.7741
Epoch 2/30
46/46 [==============================] - 4s 88ms/step - loss: 0.4754 - binary_accuracy: 0.7924 - val_loss: 0.4471 - val_binary_accuracy: 0.7941
Epoch 3/30
46/46 [==============================] - 3s 75ms/step - loss: 0.4079 - binary_accuracy: 0.8267 - val_loss: 0.4587 - val_binary_accuracy: 0.7968
Epoch 4/30
46/46 [==============================] - 3s 70ms/step - loss: 0.3804 - binary_accuracy: 0.8382 - val_loss: 0.4876 - val_binary_accuracy: 0.7941
Epoch 5/30
46/46 [==============================] - 4s 78ms/step - loss: 0.3692 - binary_accuracy: 0.8468 - val_loss: 0.5460 - val_binary_accuracy: 0.7901
Epoch 6/30
46/46 [==============================] - 4s 81ms/step - loss: 0.3697 - binary_accuracy: 0.8523 - val_loss: 0.5506 - val_binary_accuracy: 0.7807
Epoch 7/30
46/46 [==============================] - 4s 96ms/step - l

In [243]:
model_lstm.save('models/model_lstm.h5')

# BERT Model (Distilbert)

Tried to finetune a model from transformers. I used distilbert rather than the original BERT as it has much faster training time. The model is said to 'reduce the size of a BERT model by 40%, while retaining 97% of its language understanding capabilities and being 60% faster'[Sanh et al., 2020].

In [244]:
# Instanciate model and tokenizer
distil_bert_tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
distil_bert_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)


# Because Bert/Distilbert have their own tokenizer, we'll need to retokenizer the dataset from the start
# However, keeping same proportion and same random state to ensure same data in each split


    # Splitting dataset 
x_train, x_test, y_train, y_test = train_test_split(df.text_clean, df.target ,test_size=0.25,random_state = 0, stratify = df.target)
x_test, x_valid, y_test, y_valid = train_test_split(x_test, y_test ,test_size=0.6,random_state = 0, stratify = y_test) # 0.6 * 0.25 = 0.15



    # Both train_encodings & val are dict-like object with both the tokens and the attention masks 
train_encodings = distil_bert_tokenizer(x_train.to_list(),
                            truncation=True,
                            padding=True)

val_encodings = distil_bert_tokenizer(x_valid.to_list(),
                            truncation=True,
                            padding=True)

x_test = distil_bert_tokenizer_2(x_test.tolist(),
                                 truncation=True,
                                 padding=True,
                                 return_tensors="tf" # Will be able to use directly for preds after
                                 ) 

    # Creating TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    y_valid
))

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_transform', 'activation_13', 'vocab_projector', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_519', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use 

In [246]:
BATCH_SIZE = 124

# Compiling model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) # Low learning rate for finetuning

distil_bert_model.compile(optimizer=optimizer, 
                            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                            metrics=['accuracy'])

# Creating an early stopper
callbacks = EarlyStopping(monitor ="val_loss", 
                                        mode ="min", patience = 2, # Lower patience as model converges much more quicklu & each epoch takes much longer
                                        restore_best_weights = True)

# Training
distil_bert_model.fit(train_dataset.shuffle(100).batch(BATCH_SIZE),
          epochs=10,
          batch_size=BATCH_SIZE,
          validation_data=val_dataset.shuffle(100).batch(BATCH_SIZE),
          callbacks = callbacks)

Epoch 1/10
46/46 [==============================] - 3820s 84s/step - loss: 0.4179 - accuracy: 0.8283 - val_loss: 0.4006 - val_accuracy: 0.8381
Epoch 2/10
46/46 [==============================] - 3314s 72s/step - loss: 0.3332 - accuracy: 0.8692 - val_loss: 0.4168 - val_accuracy: 0.8274
Epoch 3/10
46/46 [==============================] - 2239s 48s/step - loss: 0.2593 - accuracy: 0.9029 - val_loss: 0.4607 - val_accuracy: 0.8301


In [248]:
# Saving model
distil_bert_model.save_pretrained("./models/distil_bert_weights") # Saved a different way as it is a subclassed model

# Model Comparison

Because the Kaggle competition is judged on F1 score, we'll use that metric to compare the models as well

In [271]:
warnings.filterwarnings('ignore')


# Load the different models
RNN_model = tf.keras.models.load_model('models/model_RNN.h5')
GRU_model = tf.keras.models.load_model('models/model_GRU.h5')
LSTM_model = tf.keras.models.load_model('models/model_lstm.h5')
Dist_BERT_model = TFDistilBertForSequenceClassification.from_pretrained("./models/distil_bert_weights")

## Setting up 
actual_values = Y_test 
threshold = 0.5

# Get the predictions
pred_RNN  = RNN_model.predict(X_test)
pred_GRU  = GRU_model.predict(X_test)
pred_LSTM = LSTM_model.predict(X_test)
pred_BERT = Dist_BERT_model.predict(x_test['input_ids'])  # because different tokenization

# Transforming output
pred_RNN = np.where(pred_RNN > threshold, 1, 0)
pred_GRU = np.where(pred_GRU > threshold, 1, 0)
pred_LSTM = np.where(pred_LSTM > threshold, 1, 0)
pred_BERT = pred_BERT['logits'].argmax(axis=1)


# Compare F1 scores
print('RNN F1 score:')
print(f1_score(actual_values,pred_RNN))

print('GRU F1 score:')
print(f1_score(actual_values,pred_GRU))

print('LSTM F1 score:')
print(f1_score(actual_values,pred_LSTM))


print('BERT F1 score:')
print(f1_score(actual_values,pred_BERT))



Some layers from the model checkpoint at ./models/distil_bert_weights were not used when initializing TFDistilBertForSequenceClassification: ['dropout_519']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./models/distil_bert_weights and are newly initialized: ['dropout_659']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RNN F1 score:
0.7278382581648523
GRU F1 score:
0.7163120567375887
LSTM F1 score:
0.7344827586206896
BERT F1 score:
0.7826086956521738


# Retraining Model on whole dataset

As the distilbert model performed the best, we'll use this one to make our final submissions. In order to avoid 'wasting' any data, we'll now retrain it with the whole dataset. 

In [275]:
# Tokenize full dataset from scratch
# Here tokenizer is already instanciated in this python file so we won't instanciate it again
ds_full =  distil_bert_tokenizer(df.text_clean.tolist(),
                            truncation=True,
                            padding=True)

# Tf dataset
train_full = tf.data.Dataset.from_tensor_slices((
    dict(ds_full),
    df.target.tolist()
))
                        

In [277]:
BATCH_SIZE = 124
# Reloading model to give it final name
Final_model = TFDistilBertForSequenceClassification.from_pretrained("./models/distil_bert_weights") 

# Compiling model 
# We use exactly the same as what was used previously
# We could have decreased learning rate further given even more finetuning but given its already fairly low we won't touch it
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) 

Final_model.compile(optimizer=optimizer, 
                            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                            metrics=['accuracy'])

# No callbacks this time as we'll train on same number of epochs as what was done previously given no validation dataset

# Training
Final_model.fit(train_full.shuffle(100).batch(BATCH_SIZE),
          epochs=2, #The model had trained over 3 epochs, however it had restored the best weights (those at epoch 2) so we'll keep it this way 
          batch_size=BATCH_SIZE,
          )

Some layers from the model checkpoint at ./models/distil_bert_weights were not used when initializing TFDistilBertForSequenceClassification: ['dropout_519']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./models/distil_bert_weights and are newly initialized: ['dropout_699']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/2
61/61 [==============================] - 2066s 33s/step - loss: 0.3760 - accuracy: 0.8465
Epoch 2/2
61/61 [==============================] - 2466s 40s/step - loss: 0.3322 - accuracy: 0.8684


In [278]:
# Saving model
Final_model.save_pretrained("./models/Final_model")

## Predictions on test dataset

In [279]:
# Loading dataset
df_test = pd.read_csv('Data/test.csv')

# Same cleaning up as what was done on train dataset
df_test['text_clean'] = df_test["text"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" "))
df_test['text_clean'] = df_test["text_clean"].apply(lambda x: x.replace(" +"," ").lower().strip())
df_test['text_clean'] = df_test["text_clean"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) and (token.text not in STOP_WORDS)]))

# Last check
df_test.head(15)

,id,keyword,location,text,text_clean
0,0,NaN,NaN,Just happened a terrible car crash,happen terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",hear earthquake different city stay safe
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",forest fire spot pond geese flee street save
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse light spokane wildfire
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kill 28 china taiwan
5,12,NaN,NaN,We're shaking...It's an earthquake,shakingit earthquake
6,21,NaN,NaN,They'd probably still show more life than Arse...,d probably life arsenal yesterday eh eh
7,22,NaN,NaN,Hey! How are you?,hey
8,27,NaN,NaN,What a nice hat?,nice hat
9,29,NaN,NaN,Fuck off!,fuck


In [292]:
# Tokenizing text
ds_test =  distil_bert_tokenizer(df_test.text_clean.tolist(),
                            truncation=True,
                            padding=True,
                            return_tensors="tf")

# Prediction
preds = Final_model.predict(ds_test['input_ids'])['logits'].argmax(axis=1)

# Saving submissions as .csv
preds_df = pd.DataFrame()
preds_df['target'] = preds
preds_df.head(15) 
preds_df.to_csv('submission.csv', index=True, index_label='id') #formatting at required on Kaggle